In [19]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import seaborn as sns
import json

In [15]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

In [16]:
# !unzip data.zip

In [20]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = datagen.flow_from_directory(
    'train_directory',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',  # For binary classification
    shuffle=True
)

In [ ]:
test_generator = datagen.flow_from_directory(
    'test_directory',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential([
    
    Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Dropout(0.25),
    Flatten(),
    Dense(64, activation='relu'),
    
    # Dropout(0.5),
    Dense(units=1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.002, momentum=0.8),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
history.history

In [ ]:
training_accuracies = history.history['accuracy']

median_accuracy = np.median(training_accuracies)

std_accuracy = np.std(training_accuracies)

print(f"Median training accuracy: {median_accuracy}")
print(f"Standard deviation of training accuracy: {std_accuracy}")

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
train_generator_aug = train_datagen.flow_from_directory(
    'train_directory',
    target_size=(200, 200),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

In [ ]:
history_aug = model.fit(
    train_generator_aug,
    epochs=10,
    validation_data=test_generator
)

In [ ]:
test_loss = history_aug.history['val_loss']
mean_test_loss = np.mean(test_loss)
print(f"Mean test loss: {mean_test_loss}")

In [ ]:
test_accuracy = history_aug.history['val_accuracy']
last_5_epochs_accuracy = test_accuracy[5:]
mean_last_5_accuracy = np.mean(last_5_epochs_accuracy)
print(f"Mean test accuracy for last 5 epochs: {mean_last_5_accuracy}")

In [ ]:
model.save('model.h5')

In [ ]:
train_loss, train_acc = model.evaluate(test_generator)

In [ ]:
print(train_loss, train_acc)

In [ ]:
with open ('training_hist.json', 'w') as f:
    json.dump(history.history,f)

In [ ]:
epochs = [i for i in range(1,11)]
plt.plot(epochs, history.history['accuracy'], color='red', label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], color='blue', label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
# plt.savefig("output.jpg")
# Saving figure by changing parameter values
# plt.savefig("output1", facecolor='y', bbox_inches="tight",
            pad_inches=0.3, transparent=True)
plt.show()

In [ ]:
epochs = [i for i in range(1,6)]
plt.plot(epochs, history.history['loss'], color='red', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], color='blue', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
# plt.savefig("outputloss2.jpg")
plt.show()